# import the necessary libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.utils import to_categorical
import torch.nn as nn
import torch.optim as optim
from transformers import AdamW
from transformers import AutoTokenizer, BertTokenizer, BertModel
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,GRU,SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Using device: cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train=pd.read_excel("/content/drive/MyDrive/NLP files-20240522T143610Z-001/NLP files/preprocessed_data_train2.xlsx")
df_test=pd.read_excel("/content/drive/MyDrive/NLP files-20240522T143610Z-001/NLP files/preprocessed_data_test2.xlsx")

In [ ]:
df_train.head()

,Type,News,processed_text
0,politics,اشتباك الحريري-عون.. اتهامات لباسيل بالتمسك با...,اشتباك حريري عون اتهام باسيل تمسك ثلث معطل بقا...
1,politics,عون: الحريري أصبح غريب الأطوار,عون حريري أصبح غريب اطوار
2,politics,وزير الخارجية الأمريكي: ندرس سحب كامل قواتنا م...,وزير خارجيه أمريكي درس سحب كامل قوة أفغانستان حل
3,politics,أفغانستان.. استعدادات حثيثة لاجتماع تركيا وكاب...,أفغانستان استعداد حثيث اجتماع تركيا كابل اتهم ...
4,politics,أندبندنت‮:‬ ‬مفاوضات ‬سرّية ‬‬كادت ‬تنقذ ‬القذافي,اندبندنت مفاوضة سر يه كاد أنقذ قذافي


In [ ]:
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder=LabelEncoder()
y_train=label_encoder.fit_transform(train_df['Type'])
y_val=label_encoder.transform(val_df['Type'])
y_test=label_encoder.transform(df_test['Type'])


y_train_one_hot_encoded=to_categorical(y_train)
y_val_one_hot_encoded=to_categorical(y_val)
y_test_one_hot_encoded=to_categorical(y_test)

# Inputs for Neural network

In [ ]:
tokenizer = Tokenizer(oov_token='<oov>') # For those words which are not found in word_index
all_text=df_train['processed_text'].tolist()
tokenizer.fit_on_texts(all_text)
total_words = len(tokenizer.word_index) + 1

print("Total number of words: ", total_words)
print("Word: ID")
print("------------")
print("<oov>: ", tokenizer.word_index['<oov>'])
print("غريب: ", tokenizer.word_index['غريب'])
print("كره: ", tokenizer.word_index['كره'])
print("اطوار: ", tokenizer.word_index['اطوار'])

Total number of words:  5972
Word: ID
------------
<oov>:  1
غريب:  2076
كره:  65
اطوار:  3539


In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_df['processed_text'])
val_sequences = tokenizer.texts_to_sequences(val_df['processed_text'])
test_sequences = tokenizer.texts_to_sequences(df_test['processed_text'])

In [ ]:
max_sequence_len = max(len(seq) for seq in train_sequences)

In [ ]:
padded_train = pad_sequences(train_sequences, maxlen=max_sequence_len, padding='pre')
padded_val = pad_sequences(val_sequences, maxlen=max_sequence_len, padding='pre',truncating='pre')
padded_test = pad_sequences(test_sequences, maxlen=max_sequence_len, padding='pre',truncating='pre')


In [ ]:
result={}

# basic encoding



## TF-IDF





In [ ]:
tf_results={}

In [ ]:
vectorizer_tf = TfidfVectorizer(max_features=300)
X_train_tfidf = vectorizer_tf.fit_transform(train_df['processed_text']).toarray()
X_val_tfidf = vectorizer_tf.fit_transform(val_df['processed_text']).toarray()
X_test_tfidf = vectorizer_tf.fit_transform(df_test['processed_text']).toarray()

tfidf_train_feature = pd.DataFrame(X_train_tfidf, columns=vectorizer_tf.get_feature_names_out())
tfidf_val_feature = pd.DataFrame(X_val_tfidf, columns=vectorizer_tf.get_feature_names_out())
tfidf_test_feature = pd.DataFrame(X_test_tfidf, columns=vectorizer_tf.get_feature_names_out())

### Traditional models

#### naive bayes


In [ ]:
clf_nb_tf =GaussianNB()
clf_nb_tf.fit(X_train_tfidf ,train_df['Type'])
y_pred_nb_val_tf = clf_nb_tf.predict(X_val_tfidf)
accuracy_nb_val_tf = accuracy_score(val_df['Type'], y_pred_nb_val_tf)
precision_nb_val_tf = precision_score(val_df['Type'], y_pred_nb_val_tf, average='weighted')
recall_nb_val_tf = recall_score(val_df['Type'], y_pred_nb_val_tf, average='weighted')
f1_nb_val_tf = f1_score(val_df['Type'], y_pred_nb_val_tf, average='weighted')

print(f"Accuracy validation: {accuracy_nb_val_tf:.2f}")
print(f"Precision validation: {precision_nb_val_tf:.2f}")
print(f"Recall validation: {recall_nb_val_tf :.2f}")
print(f"F1 Score validation: {f1_nb_val_tf:.2f}")
print("Classification Report for validation:")
print(classification_report(val_df['Type'], y_pred_nb_val_tf))

tf_results['Naive Bayes Accuracy tf val']=accuracy_nb_val_tf
tf_results['Naive Bayes Precision tf val']=precision_nb_val_tf
tf_results['Naive Bayes Recall tf val']=recall_nb_val_tf
tf_results['Naive Bayes F1 tf val']=f1_nb_val_tf


Accuracy validation: 0.41
Precision validation: 0.52
Recall validation: 0.41
F1 Score validation: 0.44
Classification Report for validation:
              precision    recall  f1-score   support

    economic       0.18      0.31      0.23       180
    politics       0.71      0.46      0.56       601
       sport       0.27      0.34      0.30       198
        tech       0.15      0.43      0.22        21

    accuracy                           0.41      1000
   macro avg       0.33      0.38      0.33      1000
weighted avg       0.52      0.41      0.44      1000



In [ ]:
y_pred_nb_test_tf = clf_nb_tf.predict(X_test_tfidf)

accuracy_nb_test_tf = accuracy_score(df_test['Type'], y_pred_nb_test_tf)
precision_nb_test_tf = precision_score(df_test['Type'], y_pred_nb_test_tf, average='weighted')
recall_nb_test_tf = recall_score(df_test['Type'], y_pred_nb_test_tf, average='weighted')
f1_nb_test_tf = f1_score(df_test['Type'], y_pred_nb_test_tf, average='weighted')

print(f"Accuracy for test: {accuracy_nb_test_tf:.2f}")
print(f"Precision for test: {precision_nb_test_tf:.2f}")
print(f"Recall for test: {recall_nb_test_tf:.2f}")
print(f"F1 Score for test: {f1_nb_test_tf:.2f}")
print("Classification Report for test:")
print(classification_report(df_test['Type'], y_pred_nb_test_tf))

tf_results['Naive Bayes Accuracy tf test']=accuracy_nb_test_tf
tf_results['Naive Bayes Precision tf test']=precision_nb_test_tf
tf_results['Naive Bayes Recall tf test']=recall_nb_test_tf
tf_results['Naive Bayes F1 tf test']=f1_nb_test_tf

Accuracy for test: 0.39
Precision for test: 0.40
Recall for test: 0.39
F1 Score for test: 0.39
Classification Report for test:
              precision    recall  f1-score   support

    economic       0.18      0.21      0.19       200
    politics       0.60      0.56      0.58       512
       sport       0.19      0.22      0.21       200
        tech       0.27      0.15      0.19        88

    accuracy                           0.39      1000
   macro avg       0.31      0.29      0.29      1000
weighted avg       0.40      0.39      0.39      1000



#### Logistic Regression

In [ ]:

from sklearn.linear_model import LogisticRegression

clf_lr_tf = LogisticRegression(random_state=0)
clf_lr_tf.fit(X_train_tfidf, train_df['Type'])

y_pred_val_lr_tf = clf_lr_tf.predict(X_val_tfidf)
accuracy_val_lr_tf = accuracy_score(val_df['Type'], y_pred_val_lr_tf )
precision_val_lr_tf = precision_score(val_df['Type'], y_pred_val_lr_tf , average='weighted')
recall_val_lr_tf = recall_score(val_df['Type'],y_pred_val_lr_tf , average='weighted')
f1_val_lr_tf = f1_score(val_df['Type'],y_pred_val_lr_tf , average='weighted')

print(f"Accuracy validation: {accuracy_val_lr_tf:.2f}")
print(f"Precision validation: {precision_val_lr_tf:.2f}")
print(f"Recall validation: {recall_val_lr_tf:.2f}")
print(f"F1 Score validation: {f1_val_lr_tf:.2f}")
print("Classification Report for validation:")
print(classification_report(val_df['Type'], y_pred_val_lr_tf ))

tf_results['Logistic Regression Accuracy tf val']=accuracy_val_lr_tf
tf_results['Logistic Regression Precision tf val']=precision_val_lr_tf
tf_results['Logistic Regression Recall tf val']=recall_val_lr_tf
tf_results['Logistic Regression F1 tf val']=f1_val_lr_tf


Accuracy validation: 0.51
Precision validation: 0.45
Recall validation: 0.51
F1 Score validation: 0.47
Classification Report for validation:
              precision    recall  f1-score   support

    economic       0.21      0.14      0.17       180
    politics       0.61      0.76      0.68       601
       sport       0.23      0.15      0.18       198
        tech       0.00      0.00      0.00        21

    accuracy                           0.51      1000
   macro avg       0.26      0.26      0.26      1000
weighted avg       0.45      0.51      0.47      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

In [ ]:
y_pred_test_lr_test = clf_lr_tf.predict(X_test_tfidf)

y_pred_test_lr_tf = clf_lr_tf.predict(X_test_tfidf)
accuracy_test_lr_tf = accuracy_score(df_test['Type'], y_pred_test_lr_tf )
precision_test_lr_tf = precision_score(df_test['Type'], y_pred_test_lr_tf , average='weighted')
recall_test_lr_tf = recall_score(df_test['Type'],y_pred_test_lr_tf , average='weighted')
f1_test_lr_tf = f1_score(df_test['Type'],y_pred_test_lr_tf , average='weighted')

print(f"Accuracy test: {accuracy_test_lr_tf:.2f}")
print(f"Precision test: {precision_test_lr_tf:.2f}")
print(f"Recall test: {recall_test_lr_tf:.2f}")
print(f"F1 Score test: {f1_test_lr_tf:.2f}")
print("Classification Report for test:")
print(classification_report(df_test['Type'], y_pred_test_lr_tf ))
tf_results['Logistic Regression Accuracy tf test']=accuracy_test_lr_tf
tf_results['Logistic Regression Precision tf test']=precision_test_lr_tf
tf_results['Logistic Regression Recall tf test']=recall_test_lr_tf
tf_results['Logistic Regression F1 tf test']=f1_test_lr_tf

Accuracy test: 0.47
Precision test: 0.37
Recall test: 0.47
F1 Score test: 0.40
Classification Report for test:
              precision    recall  f1-score   support

    economic       0.29      0.15      0.20       200
    politics       0.53      0.80      0.64       512
       sport       0.22      0.13      0.16       200
        tech       0.00      0.00      0.00        88

    accuracy                           0.47      1000
   macro avg       0.26      0.27      0.25      1000
weighted avg       0.37      0.47      0.40      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

### Neural Networks

#### embeddings_weights

In [ ]:
vectorizer_n_tf = TfidfVectorizer(max_features=300)
vectorizer_n_tf.fit(all_text)

TfidfVectorizer(max_features=300)

In [ ]:
tfidf_max_features=vectorizer_n_tf.max_features

In [ ]:
tfidf_embedding_matrix=np.zeros((total_words,tfidf_max_features))
for word,i in tokenizer.word_index.items():
  if word in vectorizer_n_tf.vocabulary_:
    tfidf_embedding=vectorizer_n_tf.transform([word]).toarray()[0]
    tfidf_embedding_matrix[i]=tfidf_embedding
  else:
    tfidf_embedding_matrix[i]=np.random.uniform(-25,25)

#### LSTM

In [ ]:
model_LSTM_tfidf=Sequential()
model_LSTM_tfidf.add(Embedding(input_dim=total_words,
                    output_dim=300,
                    input_length=max_sequence_len
                    ,weights=[tfidf_embedding_matrix],trainable=False))
model_LSTM_tfidf.add(LSTM(100))
model_LSTM_tfidf.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_LSTM_tfidf.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_LSTM_tfidf.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)

Epoch 1/10
125/125 [==============================] - 4s 4ms/step - loss: 0.9896 - accuracy: 0.6212
Epoch 2/10
125/125 [==============================] - 1s 4ms/step - loss: 0.7725 - accuracy: 0.7035
Epoch 3/10
125/125 [==============================] - 1s 4ms/step - loss: 0.6588 - accuracy: 0.7460
Epoch 4/10
125/125 [==============================] - 1s 4ms/step - loss: 0.6081 - accuracy: 0.7753
Epoch 5/10
125/125 [==============================] - 1s 4ms/step - loss: 0.5397 - accuracy: 0.8043
Epoch 6/10
125/125 [==============================] - 1s 4ms/step - loss: 0.5217 - accuracy: 0.8085
Epoch 7/10
125/125 [==============================] - 1s 4ms/step - loss: 0.4926 - accuracy: 0.8123
Epoch 8/10
125/125 [==============================] - 1s 4ms/step - loss: 0.4830 - accuracy: 0.8240
Epoch 9/10
125/125 [==============================] - 1s 4ms/step - loss: 0.4480 - accuracy: 0.8332
Epoch 10/10
125/125 [==============================] - 1s 4ms/step - loss: 0.4289 - accuracy: 0.8405

In [ ]:
val_loss_tf_lstm, val_accuracy_tf_lstm = model_LSTM_tfidf.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'Validation Accuracy: {val_accuracy_tf_lstm}')

y_pred = model_LSTM_tfidf.predict(padded_val)
y_pred_lstm_tf_val = np.argmax(y_pred, axis=1)
accuracy_lstm_tf_val = accuracy_score(y_val, y_pred_lstm_tf_val)
precision_lstm_tf_val = precision_score(y_val ,y_pred_lstm_tf_val, average='weighted')
recall_lstm_tf_val = recall_score(y_val, y_pred_lstm_tf_val, average='weighted')
f1_lstm_tf_val = f1_score(y_val, y_pred_lstm_tf_val, average='weighted')
print(f'Accuracy: {accuracy_lstm_tf_val:.4f}')
print(f'Precision: {precision_lstm_tf_val:.4f}')
print(f'Recall: {recall_lstm_tf_val:.4f}')
print(f'F1 Score: {f1_lstm_tf_val:.4f}')
print("Classification Report:")
print(classification_report(y_val, y_pred_lstm_tf_val))
tf_results['LSTM Accuracy tf val']=accuracy_lstm_tf_val
tf_results['LSTM Precision tf val']=precision_lstm_tf_val
tf_results['LSTM Recall tf val']=recall_lstm_tf_val
tf_results['LSTM F1 tf val']=f1_lstm_tf_val


32/32 [==============================] - 1s 5ms/step - loss: 0.7350 - accuracy: 0.7500
Validation Accuracy: 0.75
32/32 [==============================] - 1s 3ms/step
Accuracy: 0.7500
Precision: 0.7406
Recall: 0.7500
F1 Score: 0.7364
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.57      0.58       180
           1       0.77      0.89      0.83       601
           2       0.87      0.57      0.69       198
           3       0.00      0.00      0.00        21

    accuracy                           0.75      1000
   macro avg       0.56      0.51      0.52      1000
weighted avg       0.74      0.75      0.74      1000



In [ ]:
test_loss_tf_lstm, test_accuracy_tf_lstm = model_LSTM_tfidf.evaluate(padded_test, y_test_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {test_accuracy_tf_lstm}')

y_pred = model_LSTM_tfidf.predict(padded_test)
y_pred_lstm_tf_test = np.argmax(y_pred, axis=1)
accuracy_lstm_tf_test = accuracy_score(y_test, y_pred_lstm_tf_test)
precision_lstm_tf_test = precision_score(y_test,y_pred_lstm_tf_test, average='weighted')
recall_lstm_tf_test = recall_score(y_test,y_pred_lstm_tf_test, average='weighted')
f1_lstm_tf_test = f1_score(y_test, y_pred_lstm_tf_test, average='weighted')

print(f'Accuracy: {accuracy_lstm_tf_test:.4f}')
print(f'Precision: {precision_lstm_tf_test:.4f}')
print(f'Recall: {recall_lstm_tf_test:.4f}')
print(f'F1 Score: {f1_lstm_tf_test:.4f}')
print("Classification Report:")
print(classification_report(y_test, y_pred_lstm_tf_test))
tf_results['LSTM Accuracy tf test']=accuracy_lstm_tf_test
tf_results['LSTM Precision tf test']=precision_lstm_tf_test
tf_results['LSTM Recall tf test']=recall_lstm_tf_test
tf_results['LSTM F1 tf test']=f1_lstm_tf_test


32/32 [==============================] - 0s 4ms/step - loss: 1.0099 - accuracy: 0.6850
Test Accuracy: 0.6850000023841858
32/32 [==============================] - 0s 3ms/step
Accuracy: 0.6850
Precision: 0.7286
Recall: 0.6850
F1 Score: 0.6596
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.60      0.55       200
           1       0.69      0.82      0.75       512
           2       0.92      0.71      0.81       200
           3       1.00      0.02      0.04        88

    accuracy                           0.69      1000
   macro avg       0.78      0.54      0.54      1000
weighted avg       0.73      0.69      0.66      1000



#### RNN

In [ ]:
model_rnn_tfidf=Sequential()
model_rnn_tfidf.add(Embedding(input_dim=total_words,
                    output_dim=300,
                    input_length=max_sequence_len
                    ,weights=[tfidf_embedding_matrix],trainable=False))
model_rnn_tfidf.add(SimpleRNN(100))
model_rnn_tfidf.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_rnn_tfidf.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_rnn_tfidf.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)

Epoch 1/10
125/125 [==============================] - 3s 12ms/step - loss: 1.2492 - accuracy: 0.5278
Epoch 2/10
125/125 [==============================] - 1s 12ms/step - loss: 1.0107 - accuracy: 0.6133
Epoch 3/10
125/125 [==============================] - 1s 12ms/step - loss: 0.9176 - accuracy: 0.6535
Epoch 4/10
125/125 [==============================] - 2s 12ms/step - loss: 0.8928 - accuracy: 0.6540
Epoch 5/10
125/125 [==============================] - 2s 12ms/step - loss: 0.8939 - accuracy: 0.6525
Epoch 6/10
125/125 [==============================] - 2s 12ms/step - loss: 0.8403 - accuracy: 0.6823
Epoch 7/10
125/125 [==============================] - 2s 16ms/step - loss: 0.8509 - accuracy: 0.6758
Epoch 8/10
125/125 [==============================] - 3s 20ms/step - loss: 0.8258 - accuracy: 0.6850
Epoch 9/10
125/125 [==============================] - 2s 15ms/step - loss: 0.8107 - accuracy: 0.6895
Epoch 10/10
125/125 [==============================] - 2s 12ms/step - loss: 0.7917 - accura

In [ ]:
val_loss_tf_rnn, val_accuracy_tf_rnn = model_rnn_tfidf.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'Validation Accuracy: {val_accuracy_tf_rnn}')
y_pred = model_rnn_tfidf.predict(padded_val)
y_pred_rnn_tf_val = np.argmax(y_pred, axis=1)
accuracy_rnn_tf_val = accuracy_score(y_val, y_pred_rnn_tf_val)
precision_rnn_tf_val = precision_score(y_val, y_pred_rnn_tf_val, average='weighted')
recall_rnn_tf_val = recall_score(y_val, y_pred_rnn_tf_val, average='weighted')
f1_rnn_tf_val = f1_score(y_val,y_pred_rnn_tf_val, average='weighted')

print(f'Accuracy: {accuracy_rnn_tf_val:.4f}')
print(f'Precision: {precision_rnn_tf_val:.4f}')
print(f'Recall: {recall_rnn_tf_val:.4f}')
print(f'F1 Score: {f1_rnn_tf_val:.4f}')
print("Classification Report:")
print(classification_report(y_val, y_pred_rnn_tf_val))
tf_results['RNN Accuracy tf val']=accuracy_rnn_tf_val
tf_results['RNN Precision tf val']=precision_rnn_tf_val
tf_results['RNN  Recall tf val']=recall_rnn_tf_val
tf_results['RNN  F1 tf val']=f1_rnn_tf_val

32/32 [==============================] - 0s 4ms/step - loss: 0.8768 - accuracy: 0.6550
Validation Accuracy: 0.6549999713897705
32/32 [==============================] - 0s 3ms/step
Accuracy: 0.6550
Precision: 0.6659
Recall: 0.6550
F1 Score: 0.6006
Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.25      0.32       180
           1       0.66      0.93      0.77       601
           2       0.94      0.25      0.39       198
           3       0.00      0.00      0.00        21

    accuracy                           0.66      1000
   macro avg       0.52      0.36      0.37      1000
weighted avg       0.67      0.66      0.60      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

In [ ]:
test_loss_tf_rnn, test_accuracy_tf_rnn = model_rnn_tfidf.evaluate(padded_test, y_test_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {test_accuracy_tf_rnn}')
y_pred = model_rnn_tfidf.predict(padded_test)
y_pred_rnn_tf_test = np.argmax(y_pred, axis=1)
accuracy_rnn_tf_test = accuracy_score(y_test, y_pred_rnn_tf_test)
precision_rnn_tf_test = precision_score(y_test, y_pred_rnn_tf_test, average='weighted')
recall_rnn_tf_test = recall_score(y_test, y_pred_rnn_tf_test, average='weighted')
f1_rnn_tf_test = f1_score(y_test,y_pred_rnn_tf_test, average='weighted')

print(f'Accuracy: {accuracy_rnn_tf_test:.4f}')
print(f'Precision: {precision_rnn_tf_test:.4f}')
print(f'Recall: {recall_rnn_tf_test:.4f}')
print(f'F1 Score: {f1_rnn_tf_test:.4f}')
print("Classification Report:")
print(classification_report(y_test, y_pred_rnn_tf_test))

tf_results['RNN Accuracy tf test']=accuracy_rnn_tf_test
tf_results['RNN Precision tf test']=precision_rnn_tf_test
tf_results['RNN  Recall tf test']=recall_rnn_tf_test
tf_results['RNN  F1 tf test']=f1_rnn_tf_test

32/32 [==============================] - 0s 4ms/step - loss: 1.1349 - accuracy: 0.5950
Test Accuracy: 0.5950000286102295
32/32 [==============================] - 0s 3ms/step
Accuracy: 0.5950
Precision: 0.5591
Recall: 0.5950
F1 Score: 0.5303
Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.20      0.27       200
           1       0.58      0.93      0.71       512
           2       0.87      0.41      0.55       200
           3       0.00      0.00      0.00        88

    accuracy                           0.59      1000
   macro avg       0.47      0.38      0.38      1000
weighted avg       0.56      0.59      0.53      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

### Evaluation of TF-IDF across Different models

In [ ]:
tf_results

{'Naive Bayes Accuracy tf val': 0.409,
 'Naive Bayes Precision tf val': 0.5162214560870199,
 'Naive Bayes Recall tf val': 0.409,
 'Naive Bayes F1 tf val': 0.4419061282952751,
 'Naive Bayes Accuracy tf test': 0.387,
 'Naive Bayes Precision tf test': 0.4038090595340811,
 'Naive Bayes Recall tf test': 0.387,
 'Naive Bayes F1 tf test': 0.3929418279937353,
 'Logistic Regression Accuracy tf val': 0.512,
 'Logistic Regression Precision tf val': 0.45003550911557744,
 'Logistic Regression Recall tf val': 0.512,
 'Logistic Regression F1 tf val': 0.4734265872685527,
 'Logistic Regression Accuracy tf test': 0.468,
 'Logistic Regression Precision tf test': 0.3734102513462627,
 'Logistic Regression Recall tf test': 0.468,
 'Logistic Regression F1 tf test': 0.39992889663995884,
 'LSTM Accuracy tf val': 0.75,
 'LSTM Precision tf val': 0.7405548022616081,
 'LSTM Recall tf val': 0.75,
 'LSTM F1 tf val': 0.7363694814728727,
 'LSTM Accuracy tf test': 0.685,
 'LSTM Precision tf test': 0.728633436965583,
 '

In [ ]:
tf_results_list=[{'performance metric & model':k,"results":v} for k,v in tf_results.items()]
tf_df_results=pd.DataFrame(tf_results_list)
tf_df_results

,performance metric & model,results
0,Naive Bayes Accuracy tf val,0.409000
1,Naive Bayes Precision tf val,0.516221
2,Naive Bayes Recall tf val,0.409000
3,Naive Bayes F1 tf val,0.441906
4,Naive Bayes Accuracy tf test,0.387000
5,Naive Bayes Precision tf test,0.403809
6,Naive Bayes Recall tf test,0.387000
7,Naive Bayes F1 tf test,0.392942
8,Logistic Regression Accuracy tf val,0.512000
9,Logistic Regression Precision tf val,0.450036


## Bag of Words

In [ ]:
bow_results={}

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = cv_uni_vec = CountVectorizer(max_features=300,ngram_range=(1,1))
X_train_uni = cv_uni_vec.fit_transform(train_df['processed_text'])
X_val_uni = cv_uni_vec.fit_transform(val_df['processed_text'])
X_test_uni = cv_uni_vec.fit_transform(df_test['processed_text'])
uni_train_feature = pd.DataFrame(X_train_uni.toarray(), columns=cv_uni_vec.get_feature_names_out())
uni_val_feature = pd.DataFrame(X_val_uni.toarray(), columns=cv_uni_vec.get_feature_names_out())
uni_test_feature = pd.DataFrame(X_test_uni.toarray(), columns=cv_uni_vec.get_feature_names_out())


### Traditional Models

#### Naive Bayes

In [ ]:
clf_nb_bow = GaussianNB()
clf_nb_bow.fit(uni_train_feature.values, y_train)

y_pred_nb_bow_val = clf_nb_bow.predict(uni_val_feature.values)

accuracy_nb_bow_val = accuracy_score(y_val, y_pred_nb_bow_val)
print(f"Accuracy: {accuracy_nb_bow_val:.2f}")
precision_nb_bow_val = precision_score(y_val, y_pred_nb_bow_val, average='weighted')
recall_nb_bow_val = recall_score(y_val, y_pred_nb_bow_val, average='weighted')
f1_nb_bow_val = f1_score(y_val, y_pred_nb_bow_val, average='weighted')

print(f'Accuracy: {accuracy_nb_bow_val:.4f}')
print(f'Precision: {precision_nb_bow_val:.4f}')
print(f'Recall: {recall_nb_bow_val:.4f}')
print(f'F1 Score: {f1_nb_bow_val:.4f}')

print("Classification Report:")
print(classification_report(y_val, y_pred_nb_bow_val))

bow_results['Naive Bayes Accuracy bow val']=accuracy_nb_bow_val
bow_results['Naive Bayes Precision bow val']=precision_nb_bow_val
bow_results['Naive Bayes  Recall bow val']=recall_nb_bow_val
bow_results['Naive Bayes  F1 bow val']=f1_nb_bow_val

Accuracy: 0.39
Accuracy: 0.3890
Precision: 0.5189
Recall: 0.3890
F1 Score: 0.4235
Classification Report:
              precision    recall  f1-score   support

           0       0.18      0.31      0.23       180
           1       0.72      0.42      0.53       601
           2       0.25      0.37      0.30       198
           3       0.15      0.43      0.22        21

    accuracy                           0.39      1000
   macro avg       0.33      0.38      0.32      1000
weighted avg       0.52      0.39      0.42      1000



In [ ]:
y_pred_nb_bow_test = clf_nb_bow.predict(uni_test_feature.values)

accuracy_nb_bow_test = accuracy_score(y_test, y_pred_nb_bow_test)
print(f"Accuracy: {accuracy_nb_bow_test:.2f}")
precision_nb_bow_test = precision_score(y_test, y_pred_nb_bow_test, average='weighted')
recall_nb_bow_test = recall_score(y_test, y_pred_nb_bow_test, average='weighted')
f1_nb_bow_test = f1_score(y_test, y_pred_nb_bow_test, average='weighted')

print(f'Accuracy: {accuracy_nb_bow_test:.4f}')
print(f'Precision: {precision_nb_bow_test:.4f}')
print(f'Recall: {recall_nb_bow_test:.4f}')
print(f'F1 Score: {f1_nb_bow_test:.4f}')

print("Classification Report:")
print(classification_report(y_test, y_pred_nb_bow_test))

bow_results['Naive Bayes Accuracy bow test']=accuracy_nb_bow_test
bow_results['Naive Bayes Precision bow test']=precision_nb_bow_test
bow_results['Naive Bayes  Recall bow test']=recall_nb_bow_test
bow_results['Naive Bayes  F1 bow test']=f1_nb_bow_test

Accuracy: 0.36
Accuracy: 0.3650
Precision: 0.3931
Recall: 0.3650
F1 Score: 0.3749
Classification Report:
              precision    recall  f1-score   support

           0       0.19      0.21      0.20       200
           1       0.58      0.51      0.54       512
           2       0.18      0.26      0.21       200
           3       0.27      0.15      0.19        88

    accuracy                           0.36      1000
   macro avg       0.30      0.28      0.29      1000
weighted avg       0.39      0.36      0.37      1000



#### Logestic Regression

In [ ]:


clf_lr_bow = LogisticRegression()
clf_lr_bow.fit(uni_train_feature.values, y_train)

y_pred_lr_bow_val = clf_lr_bow.predict(uni_val_feature.values)

accuracy_lr_bow_val = accuracy_score(y_val, y_pred_lr_bow_val)
print(f"Accuracy: {accuracy_lr_bow_val:.2f}")
precision_lr_bow_val = precision_score(y_val, y_pred_lr_bow_val, average='weighted')
recall_lr_bow_val = recall_score(y_val, y_pred_lr_bow_val, average='weighted')
f1_lr_bow_val = f1_score(y_val, y_pred_lr_bow_val, average='weighted')

print(f'Accuracy: {accuracy_lr_bow_val:.4f}')
print(f'Precision: {precision_lr_bow_val:.4f}')
print(f'Recall: {recall_lr_bow_val:.4f}')
print(f'F1 Score: {f1_lr_bow_val:.4f}')

print("Classification Report:")
print(classification_report(y_val, y_pred_lr_bow_val))

bow_results['Logistic Regression Accuracy bow val']=accuracy_lr_bow_val
bow_results['Logistic Regression Precision bow val']=precision_lr_bow_val
bow_results['Logistic Regression  Recall bow val']=recall_lr_bow_val
bow_results['Logistic Regression  F1 bow val']=f1_lr_bow_val



Accuracy: 0.48
Accuracy: 0.4810
Precision: 0.4421
Recall: 0.4810
F1 Score: 0.4590
Classification Report:
              precision    recall  f1-score   support

           0       0.21      0.18      0.20       180
           1       0.60      0.69      0.64       601
           2       0.21      0.16      0.18       198
           3       0.00      0.00      0.00        21

    accuracy                           0.48      1000
   macro avg       0.26      0.26      0.26      1000
weighted avg       0.44      0.48      0.46      1000



In [ ]:
y_pred_lr_bow_test = clf_lr_bow.predict(uni_test_feature.values)

accuracy_lr_bow_test = accuracy_score(y_test, y_pred_lr_bow_test)
print(f"Accuracy: {accuracy_lr_bow_test:.2f}")
precision_lr_bow_test = precision_score(y_test, y_pred_lr_bow_test, average='weighted')
recall_lr_bow_test = recall_score(y_test, y_pred_lr_bow_test, average='weighted')
f1_lr_bow_test = f1_score(y_test, y_pred_lr_bow_test, average='weighted')

print(f'Accuracy: {accuracy_lr_bow_test:.4f}')
print(f'Precision: {precision_lr_bow_test:.4f}')
print(f'Recall: {recall_lr_bow_test:.4f}')
print(f'F1 Score: {f1_lr_bow_test:.4f}')

print("Classification Report:")
print(classification_report(y_test, y_pred_lr_bow_test))

bow_results['Logistic Regression Accuracy bow test']=accuracy_lr_bow_test
bow_results['Logistic Regression Precision bow test']=precision_lr_bow_test
bow_results['Logistic Regression  Recall bow test']=recall_lr_bow_test
bow_results['Logistic Regression  F1 bow test']=f1_lr_bow_test


Accuracy: 0.42
Accuracy: 0.4210
Precision: 0.3527
Recall: 0.4210
F1 Score: 0.3786
Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.17      0.20       200
           1       0.52      0.69      0.59       512
           2       0.19      0.17      0.18       200
           3       0.00      0.00      0.00        88

    accuracy                           0.42      1000
   macro avg       0.24      0.26      0.24      1000
weighted avg       0.35      0.42      0.38      1000



### Neural network

#### embedding_weights

In [ ]:

vectorizer = cv_uni_vec = CountVectorizer(max_features=300,ngram_range=(1,1))
vectorizer.fit(all_text)
bow_max_features=vectorizer.max_features
bow_embedding_matrix=np.zeros((total_words,bow_max_features))
for word,i in tokenizer.word_index.items():
  if word in vectorizer.vocabulary_:
    bow_embedding=vectorizer.transform([word]).toarray()[0]
    bow_embedding_matrix[i]=bow_embedding
  else:
    bow_embedding_matrix[i]=np.random.uniform(-25,25)


#### LSTM

In [ ]:

model_LSTM_bow=Sequential()
model_LSTM_bow.add(Embedding(input_dim=total_words,
                    output_dim=300,
                    input_length=max_sequence_len
                    ,weights=[bow_embedding_matrix],trainable=False))
model_LSTM_bow.add(LSTM(100))
model_LSTM_bow.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_LSTM_bow.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_LSTM_bow.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)


Epoch 1/10
125/125 [==============================] - 2s 5ms/step - loss: 0.9703 - accuracy: 0.6317
Epoch 2/10
125/125 [==============================] - 1s 6ms/step - loss: 0.7124 - accuracy: 0.7308
Epoch 3/10
125/125 [==============================] - 1s 5ms/step - loss: 0.5973 - accuracy: 0.7790
Epoch 4/10
125/125 [==============================] - 1s 6ms/step - loss: 0.5303 - accuracy: 0.8027
Epoch 5/10
125/125 [==============================] - 1s 6ms/step - loss: 0.5005 - accuracy: 0.8175
Epoch 6/10
125/125 [==============================] - 1s 6ms/step - loss: 0.4571 - accuracy: 0.8292
Epoch 7/10
125/125 [==============================] - 1s 5ms/step - loss: 0.4487 - accuracy: 0.8460
Epoch 8/10
125/125 [==============================] - 1s 4ms/step - loss: 0.4369 - accuracy: 0.8447
Epoch 9/10
125/125 [==============================] - 1s 4ms/step - loss: 0.4325 - accuracy: 0.8428
Epoch 10/10
125/125 [==============================] - 1s 4ms/step - loss: 0.4208 - accuracy: 0.8455

In [ ]:
val_loss_bow_lstm, val_accuracy_bow_lstm = model_LSTM_bow.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'Validation Accuracy: {val_accuracy_bow_lstm}')
y_pred = model_LSTM_bow.predict(padded_val)
y_pred_lstm_bow_val = np.argmax(y_pred, axis=1)
accuracy_lstm_bow_val = accuracy_score(y_val, y_pred_lstm_bow_val)
precision_lstm_bow_val = precision_score(y_val, y_pred_lstm_bow_val, average='weighted')
recall_lstm_bow_val = recall_score(y_val, y_pred_lstm_bow_val, average='weighted')
f1_lstm_bow_val = f1_score(y_val, y_pred_lstm_bow_val, average='weighted')

print(f'Accuracy: {accuracy_lstm_bow_val:.4f}')
print(f'Precision: {precision_lstm_bow_val:.4f}')
print(f'Recall: {recall_lstm_bow_val:.4f}')
print(f'F1 Score: {f1_lstm_bow_val:.4f}')

print("Classification Report:")
print(classification_report(y_val, y_pred_lstm_bow_val))
bow_results['LSTM Accuracy bow val']=accuracy_lstm_bow_val
bow_results['LSTM Precision bow val']=precision_lstm_bow_val
bow_results['LSTM  Recall bow val']=recall_lstm_bow_val
bow_results['LSTM  F1 bow val']=f1_lstm_bow_val



32/32 [==============================] - 0s 3ms/step - loss: 0.7121 - accuracy: 0.7560
Validation Accuracy: 0.7559999823570251
32/32 [==============================] - 0s 2ms/step
Accuracy: 0.7560
Precision: 0.7410
Recall: 0.7560
F1 Score: 0.7360
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.50      0.61       180
           1       0.75      0.92      0.83       601
           2       0.75      0.57      0.65       198
           3       0.00      0.00      0.00        21

    accuracy                           0.76      1000
   macro avg       0.57      0.50      0.52      1000
weighted avg       0.74      0.76      0.74      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

In [ ]:
test_loss_bow_lstm, test_accuracy_bow_lstm = model_LSTM_bow.evaluate(padded_test, y_test_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {test_accuracy_bow_lstm}')
y_pred = model_LSTM_bow.predict(padded_test)
y_pred_lstm_bow_test = np.argmax(y_pred, axis=1)
accuracy_lstm_bow_test = accuracy_score(y_test, y_pred_lstm_bow_test)
precision_lstm_bow_test = precision_score(y_test, y_pred_lstm_bow_test, average='weighted')
recall_lstm_bow_test = recall_score(y_test, y_pred_lstm_bow_test, average='weighted')
f1_lstm_bow_test = f1_score(y_test, y_pred_lstm_bow_test, average='weighted')

print(f'Accuracy: {accuracy_lstm_bow_test:.4f}')
print(f'Precision: {precision_lstm_bow_test:.4f}')
print(f'Recall: {recall_lstm_bow_test:.4f}')
print(f'F1 Score: {f1_lstm_bow_test:.4f}')
print("Classification Report:")
print(classification_report(y_test, y_pred_lstm_bow_test))
bow_results['LSTM Accuracy bow test']=accuracy_lstm_bow_test
bow_results['LSTM Precision bow test']=precision_lstm_bow_test
bow_results['LSTM  Recall bow test']=recall_lstm_bow_test
bow_results['LSTM  F1 bow test']=f1_lstm_bow_test


32/32 [==============================] - 0s 3ms/step - loss: 1.0506 - accuracy: 0.7140
Test Accuracy: 0.7139999866485596
32/32 [==============================] - 0s 2ms/step
Accuracy: 0.7140
Precision: 0.6986
Recall: 0.7140
F1 Score: 0.6757
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.49      0.56       200
           1       0.70      0.91      0.79       512
           2       0.85      0.73      0.79       200
           3       0.50      0.01      0.02        88

    accuracy                           0.71      1000
   macro avg       0.67      0.54      0.54      1000
weighted avg       0.70      0.71      0.68      1000



#### RNN

In [ ]:
model_rnn_bow=Sequential()
model_rnn_bow.add(Embedding(input_dim=total_words,
                    output_dim=300,
                    input_length=max_sequence_len
                    ,weights=[bow_embedding_matrix],trainable=False))
model_rnn_bow.add(SimpleRNN(100))
model_rnn_bow.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_rnn_bow.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_rnn_bow.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)


Epoch 1/10
125/125 [==============================] - 3s 12ms/step - loss: 1.2026 - accuracy: 0.5242
Epoch 2/10
125/125 [==============================] - 2s 12ms/step - loss: 0.9778 - accuracy: 0.6325
Epoch 3/10
125/125 [==============================] - 2s 12ms/step - loss: 0.8853 - accuracy: 0.6628
Epoch 4/10
125/125 [==============================] - 2s 14ms/step - loss: 0.8697 - accuracy: 0.6672
Epoch 5/10
125/125 [==============================] - 3s 20ms/step - loss: 0.8335 - accuracy: 0.6795
Epoch 6/10
125/125 [==============================] - 2s 16ms/step - loss: 0.8302 - accuracy: 0.6867
Epoch 7/10
125/125 [==============================] - 1s 12ms/step - loss: 0.8520 - accuracy: 0.6643
Epoch 8/10
125/125 [==============================] - 1s 12ms/step - loss: 0.8226 - accuracy: 0.6888
Epoch 9/10
125/125 [==============================] - 1s 12ms/step - loss: 0.7984 - accuracy: 0.7070
Epoch 10/10
125/125 [==============================] - 1s 12ms/step - loss: 0.7944 - accura

In [ ]:
val_loss_bow_rnn, val_accuracy_bow_rnn = model_rnn_bow.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'Validation Accuracy: {val_accuracy_bow_rnn}')
y_pred = model_rnn_bow.predict(padded_val)
y_pred_rnn_bow_val = np.argmax(y_pred, axis=1)
accuracy_rnn_bow_val = accuracy_score(y_val, y_pred_rnn_bow_val)
precision_rnn_bow_val = precision_score(y_val, y_pred_rnn_bow_val, average='weighted')
recall_rnn_bow_val = recall_score(y_val,y_pred_rnn_bow_val, average='weighted')
f1_rnn_bow_val = f1_score(y_val, y_pred_rnn_bow_val, average='weighted')

print(f'Accuracy: {accuracy_rnn_bow_val:.4f}')
print(f'Precision: {precision_rnn_bow_val:.4f}')
print(f'Recall: {recall_rnn_bow_val:.4f}')
print(f'F1 Score: {f1_rnn_bow_val:.4f}')
print("Classification Report:")
print(classification_report(y_val, y_pred_rnn_bow_val))
bow_results['RNN Accuracy bow val']=accuracy_rnn_bow_val
bow_results['RNN Precision bow val']=precision_rnn_bow_val
bow_results['RNN  Recall bow val']=recall_rnn_bow_val
bow_results['RNN  F1 bow val']=f1_rnn_bow_val

32/32 [==============================] - 0s 6ms/step - loss: 0.8695 - accuracy: 0.6730
Validation Accuracy: 0.6729999780654907
32/32 [==============================] - 0s 5ms/step
Accuracy: 0.6730
Precision: 0.6714
Recall: 0.6730
F1 Score: 0.6230
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.32      0.41       180
           1       0.67      0.94      0.78       601
           2       0.81      0.26      0.39       198
           3       0.00      0.00      0.00        21

    accuracy                           0.67      1000
   macro avg       0.52      0.38      0.40      1000
weighted avg       0.67      0.67      0.62      1000



In [ ]:
test_loss_bow_rnn, test_accuracy_bow_rnn = model_rnn_bow.evaluate(padded_test, y_val_one_hot_encoded, verbose=1)
print(f'test Accuracy: {test_accuracy_bow_rnn}')
y_pred = model_rnn_bow.predict(padded_test)
y_pred_rnn_bow_test = np.argmax(y_pred, axis=1)
accuracy_rnn_bow_test = accuracy_score(y_test, y_pred_rnn_bow_test)
precision_rnn_bow_test = precision_score(y_test, y_pred_rnn_bow_test, average='weighted')
recall_rnn_bow_test = recall_score(y_test,y_pred_rnn_bow_test, average='weighted')
f1_rnn_bow_test = f1_score(y_test, y_pred_rnn_bow_test, average='weighted')

print(f'Accuracy: {accuracy_rnn_bow_test:.4f}')
print(f'Precision: {precision_rnn_bow_test:.4f}')
print(f'Recall: {recall_rnn_bow_test:.4f}')
print(f'F1 Score: {f1_rnn_bow_test:.4f}')
print("Classification Report:")
print(classification_report(y_test, y_pred_rnn_bow_test))
bow_results['RNN Accuracy bow test']=accuracy_rnn_bow_test
bow_results['RNN Precision bow test']=precision_rnn_bow_test
bow_results['RNN  Recall bow test']=recall_rnn_bow_test
bow_results['RNN  F1 bow test']=f1_rnn_bow_test

32/32 [==============================] - 0s 6ms/step - loss: 1.6048 - accuracy: 0.5250
test Accuracy: 0.5249999761581421
32/32 [==============================] - 0s 5ms/step
Accuracy: 0.6120
Precision: 0.5815
Recall: 0.6120
F1 Score: 0.5506
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.26      0.35       200
           1       0.59      0.94      0.73       512
           2       0.84      0.41      0.55       200
           3       0.00      0.00      0.00        88

    accuracy                           0.61      1000
   macro avg       0.50      0.40      0.41      1000
weighted avg       0.58      0.61      0.55      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

### Evaluation of Bag of Words across the models

In [ ]:
bow_results_list=[{'performance metric & model':k,"results":v} for k,v in bow_results.items()]

bow_df_results=pd.DataFrame(bow_results_list)
bow_df_results

,performance metric & model,results
0,Naive Bayes Accuracy bow val,0.389000
1,Naive Bayes Precision bow val,0.518858
2,Naive Bayes Recall bow val,0.389000
3,Naive Bayes F1 bow val,0.423484
4,Naive Bayes Accuracy bow test,0.365000
5,Naive Bayes Precision bow test,0.393071
6,Naive Bayes Recall bow test,0.365000
7,Naive Bayes F1 bow test,0.374876
8,Logistic Regression Accuracy bow val,0.481000
9,Logistic Regression Precision bow val,0.442147


# Independent word embedding

## Word2vec twitter CBOW 300

In [ ]:
w2v_cbow_results={}

In [ ]:
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_300_twitter.zip"

--2024-06-12 00:41:16--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_300_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3325529808 (3.1G) [application/zip]
Saving to: ‘full_grams_cbow_300_twitter.zip’

full_grams_cbow_300 100%[===================>]   3.10G  65.8MB/s    in 56s     

2024-06-12 00:42:13 (56.3 MB/s) - ‘full_grams_cbow_300_twitter.zip’ saved [3325529808/3325529808]



In [ ]:
!unzip "/content/full_grams_cbow_300_twitter.zip"

Archive:  /content/full_grams_cbow_300_twitter.zip
  inflating: full_grams_cbow_300_twitter.mdl  
  inflating: full_grams_cbow_300_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_grams_cbow_300_twitter.mdl.wv.vectors.npy  


In [ ]:
import gensim
import re
import numpy as np
from nltk import ngrams

w2v_model_cbow = gensim.models.Word2Vec.load('/content/full_grams_cbow_300_twitter.mdl')

In [ ]:
OOV_tokens = []
train_tokens = []
test_tokens = []
val_tokens=[]
train_w2v_embeddings = []
test_w2v_embeddings = []

def get_doc_vec(sent, model, data):
  w2v_embeddings = []
  tokens = sent.split()
  for word in tokens:
    try:
      if data == 'train':
        w2v_embeddings.append(model.wv[word])
        train_tokens.append(word)
      elif data=='val':
        w2v_embeddings.append(model.wv[word])
        val_tokens.append(word)
      else:
        w2v_embeddings.append(model.wv[word])
        test_tokens.append(word)
    except:
      #print(word, 'does not exist in the model.')
      OOV_tokens.append(word) #if found any -> go back to pre-processing the data
      continue
  if len(w2v_embeddings) == 0:
    return None
  return sum(w2v_embeddings)/len(w2v_embeddings)

In [ ]:
X_train_w2v_embeddings = train_df['processed_text'].apply(lambda sent: get_doc_vec(sent, w2v_model_cbow,'train'))
X_val_w2v_embeddings = val_df['processed_text'].apply(lambda sent: get_doc_vec(sent, w2v_model_cbow,'val'))

X_test_w2v_embeddings = df_test['processed_text'].apply(lambda sent: get_doc_vec(sent, w2v_model_cbow,'test'))

In [ ]:
X_train_w2v_embeddings_list = []
for embedding in X_train_w2v_embeddings:
    if embedding is not None:
        X_train_w2v_embeddings_list.append(embedding)
    else:
        # Handle cases where the embedding is None
        X_train_w2v_embeddings_list.append(np.zeros_like(X_train_w2v_embeddings[0]))

X_val_w2v_embeddings_list = []
for embedding in X_val_w2v_embeddings:
    if embedding is not None:
        X_val_w2v_embeddings_list.append(embedding)
    else:
        # Handle cases where the embedding is None
        X_val_w2v_embeddings_list.append(np.zeros_like(X_val_w2v_embeddings[0]))

X_test_w2v_embeddings_list = []
for embedding in X_test_w2v_embeddings:
    if embedding is not None:
        X_test_w2v_embeddings_list.append(embedding)
    else:
        # Handle cases where the embedding is None
        X_test_w2v_embeddings_list.append(np.zeros_like(X_test_w2v_embeddings[0]))


### Traditional models

#### Naive Bayes

In [ ]:
clf_w2v_cbow_nb = GaussianNB()
clf_w2v_cbow_nb.fit(X_train_w2v_embeddings_list,train_df['Type'])

y_pred_w2v_cbow_nb_val = clf_w2v_cbow_nb.predict(X_val_w2v_embeddings_list)

accuracy_w2v_cbow_nb_val = accuracy_score(val_df['Type'], y_pred_w2v_cbow_nb_val)
precision_w2v_cbow_nb_val = precision_score(val_df['Type'], y_pred_w2v_cbow_nb_val, average='weighted')
recall_w2v_cbow_nb_val = recall_score(val_df['Type'], y_pred_w2v_cbow_nb_val, average='weighted')
f1_w2v_cbow_nb_val = f1_score(val_df['Type'], y_pred_w2v_cbow_nb_val, average='weighted')

print(f"Accuracy validation: {accuracy_w2v_cbow_nb_val:.2f}")
print(f"Precision validation: {precision_w2v_cbow_nb_val:.2f}")
print(f"Recall validation: {recall_w2v_cbow_nb_val:.2f}")
print(f"F1 Score validation: {f1_w2v_cbow_nb_val:.2f}")
print("Classification Report for validation:")
print(classification_report(val_df['Type'], y_pred_w2v_cbow_nb_val))

w2v_cbow_results['Naive Bayes Accuracy w2v cbow val']=accuracy_w2v_cbow_nb_val
w2v_cbow_results['Naive Bayes Precision w2v cbow val']=precision_w2v_cbow_nb_val
w2v_cbow_results['Naive Bayes  Recall w2v cbow val']=recall_w2v_cbow_nb_val
w2v_cbow_results['Naive Bayes  F1 w2v cbow val']=f1_w2v_cbow_nb_val

Accuracy validation: 0.86
Precision validation: 0.88
Recall validation: 0.86
F1 Score validation: 0.87
Classification Report for validation:
              precision    recall  f1-score   support

    economic       0.71      0.78      0.74       180
    politics       0.92      0.88      0.90       601
       sport       0.97      0.89      0.93       198
        tech       0.41      0.81      0.55        21

    accuracy                           0.86      1000
   macro avg       0.75      0.84      0.78      1000
weighted avg       0.88      0.86      0.87      1000



In [ ]:
y_pred_w2v_cbow_nb_test = clf_w2v_cbow_nb.predict(X_val_w2v_embeddings_list)

accuracy_w2v_cbow_nb_test = accuracy_score(df_test['Type'], y_pred_w2v_cbow_nb_test)
precision_w2v_cbow_nb_test = precision_score(df_test['Type'], y_pred_w2v_cbow_nb_test, average='weighted')
recall_w2v_cbow_nb_test = recall_score(df_test['Type'], y_pred_w2v_cbow_nb_test, average='weighted')
f1_w2v_cbow_nb_test = f1_score(df_test['Type'], y_pred_w2v_cbow_nb_test, average='weighted')

print(f"Accuracy : {accuracy_w2v_cbow_nb_test:.2f}")
print(f"Precision : {precision_w2v_cbow_nb_test:.2f}")
print(f"Recall : {recall_w2v_cbow_nb_test:.2f}")
print(f"F1 Score : {f1_w2v_cbow_nb_test:.2f}")
print("Classification Report for test:")
print(classification_report(df_test['Type'], y_pred_w2v_cbow_nb_test))

w2v_cbow_results['Naive Bayes Accuracy w2v cbow test']=accuracy_w2v_cbow_nb_test
w2v_cbow_results['Naive Bayes Precision w2v cbow test']=precision_w2v_cbow_nb_test
w2v_cbow_results['Naive Bayes  Recall w2v cbow test']=recall_w2v_cbow_nb_test
w2v_cbow_results['Naive Bayes  F1 w2v cbow test']=f1_w2v_cbow_nb_test

Accuracy : 0.36
Precision : 0.33
Recall : 0.36
F1 Score : 0.34
Classification Report for test:
              precision    recall  f1-score   support

    economic       0.18      0.17      0.18       200
    politics       0.51      0.57      0.54       512
       sport       0.16      0.14      0.15       200
        tech       0.05      0.02      0.03        88

    accuracy                           0.36      1000
   macro avg       0.22      0.23      0.22      1000
weighted avg       0.33      0.36      0.34      1000



"\nw2v_cbow_results['Naive Bayes Accuracy w2v cbow test']=accuracy_w2v_cbow_nb_test\nw2v_cbow_results['Naive Bayes Precision w2v cbow test']=precision_w2v_cbow_nb_test\nw2v_cbow_results['Naive Bayes  Recall w2v cbow test']=recall_w2v_cbow_nb_test\nw2v_cbow_results['Naive Bayes  F1 w2v cbow test']=f1_w2v_cbow_nb_test"

#### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_w2v_cbow_lr = LogisticRegression(random_state=0)
clf_w2v_cbow_lr.fit(X_train_w2v_embeddings_list, train_df['Type'])

y_pred_w2v_cbow_lr_val = clf_w2v_cbow_lr.predict(X_val_w2v_embeddings_list)

accuracy_w2v_cbow_lr_val = accuracy_score(val_df['Type'], y_pred_w2v_cbow_lr_val)
precision_w2v_cbow_lr_val = precision_score(val_df['Type'], y_pred_w2v_cbow_lr_val, average='weighted')
recall_w2v_cbow_lr_val = recall_score(val_df['Type'], y_pred_w2v_cbow_lr_val, average='weighted')
f1_w2v_cbow_lr_val = f1_score(val_df['Type'], y_pred_w2v_cbow_lr_val, average='weighted')

print(f"Accuracy validation: {accuracy_w2v_cbow_lr_val:.2f}")
print(f"Precision validation: {precision_w2v_cbow_lr_val:.2f}")
print(f"Recall validation: {recall_w2v_cbow_lr_val:.2f}")
print(f"F1 Score validation: {f1_w2v_cbow_lr_val:.2f}")
print("Classification Report for validation:")
print(classification_report(val_df['Type'], y_pred_w2v_cbow_lr_val))

w2v_cbow_results['Logistic Regression Accuracy w2v cbow val']=accuracy_w2v_cbow_lr_val
w2v_cbow_results['Logistic Regression Precision w2v cbow val']=precision_w2v_cbow_lr_val
w2v_cbow_results['Logistic Regression  Recall w2v cbow val']=recall_w2v_cbow_lr_val
w2v_cbow_results['Logistic Regression  F1 w2v cbow val']=f1_w2v_cbow_lr_val

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy validation: 0.91
Precision validation: 0.90
Recall validation: 0.91
F1 Score validation: 0.90
Classification Report for validation:
              precision    recall  f1-score   support

    economic       0.83      0.78      0.81       180
    politics       0.92      0.95      0.93       601
       sport       0.94      0.92      0.93       198
        tech       0.70      0.67      0.68        21

    accuracy                           0.91      1000
   macro avg       0.85      0.83      0.84      1000
weighted avg       0.90      0.91      0.90      1000



In [ ]:
y_pred_w2v_cbow_lr_test = clf_w2v_cbow_lr.predict(X_test_w2v_embeddings_list)

accuracy_w2v_cbow_lr_test = accuracy_score(val_df['Type'], y_pred_w2v_cbow_lr_test)
precision_w2v_cbow_lr_test = precision_score(val_df['Type'], y_pred_w2v_cbow_lr_test, average='weighted')
recall_w2v_cbow_lr_test = recall_score(val_df['Type'], y_pred_w2v_cbow_lr_test, average='weighted')
f1_w2v_cbow_lr_test = f1_score(val_df['Type'], y_pred_w2v_cbow_lr_test, average='weighted')

print(f"Accuracy : {accuracy_w2v_cbow_lr_test:.2f}")
print(f"Precision : {precision_w2v_cbow_lr_test:.2f}")
print(f"Recall : {recall_w2v_cbow_lr_test:.2f}")
print(f"F1 Score : {f1_w2v_cbow_lr_test:.2f}")
print("Classification Report for test:")
print(classification_report(val_df['Type'], y_pred_w2v_cbow_lr_test))


w2v_cbow_results['Logistic Regression Accuracy w2v cbow test']=accuracy_w2v_cbow_lr_test
w2v_cbow_results['Logistic Regression Precision w2v cbow test']=precision_w2v_cbow_lr_test
w2v_cbow_results['Logistic Regression  Recall w2v cbow test']=recall_w2v_cbow_lr_test
w2v_cbow_results['Logistic Regression  F1 w2v cbow test']=f1_w2v_cbow_lr_test

Accuracy : 0.40
Precision : 0.42
Recall : 0.39
F1 Score : 0.41
Classification Report for test:
              precision    recall  f1-score   support

    economic       0.16      0.16      0.16       180
    politics       0.60      0.56      0.58       601
       sport       0.14      0.15      0.15       198
        tech       0.02      0.05      0.03        21

    accuracy                           0.40      1000
   macro avg       0.23      0.23      0.23      1000
weighted avg       0.42      0.39      0.41      1000



### Neural Network

#### embedding weights

In [ ]:
embedding_weight_matrix_w2v_cbow=np.zeros((total_words,300))

for word, i in tokenizer.word_index.items():
  if word in w2v_model_cbow.wv:
    embedding_weight_matrix_w2v_cbow[i]=w2v_model_cbow.wv[word]
  else:
    embedding_weight_matrix_w2v_cbow[i]=np.random.uniform(-25,25)

#### LSTM

In [ ]:
model_LSTM_w2v_cbow=Sequential()
model_LSTM_w2v_cbow.add(Embedding(input_dim=total_words,
                    output_dim=300,
                    input_length=max_sequence_len
                    ,weights=[embedding_weight_matrix_w2v_cbow],trainable=False))
model_LSTM_w2v_cbow.add(LSTM(100))
model_LSTM_w2v_cbow.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_LSTM_w2v_cbow.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_LSTM_w2v_cbow.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)

Epoch 1/10
125/125 [==============================] - 3s 6ms/step - loss: 0.4615 - accuracy: 0.8307
Epoch 2/10
125/125 [==============================] - 1s 5ms/step - loss: 0.2578 - accuracy: 0.9087
Epoch 3/10
125/125 [==============================] - 1s 4ms/step - loss: 0.1952 - accuracy: 0.9312
Epoch 4/10
125/125 [==============================] - 1s 4ms/step - loss: 0.1507 - accuracy: 0.9498
Epoch 5/10
125/125 [==============================] - 1s 4ms/step - loss: 0.1032 - accuracy: 0.9647
Epoch 6/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0870 - accuracy: 0.9690
Epoch 7/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0769 - accuracy: 0.9728
Epoch 8/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0726 - accuracy: 0.9747
Epoch 9/10
125/125 [==============================] - 0s 4ms/step - loss: 0.0733 - accuracy: 0.9732
Epoch 10/10
125/125 [==============================] - 1s 4ms/step - loss: 0.1003 - accuracy: 0.9647

In [ ]:
val_loss_lstm_w2v_cbow, val_accuracy_lstm_w2v_cbow= model_LSTM_w2v_cbow.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {val_accuracy_lstm_w2v_cbow}')

y_pred =model_LSTM_w2v_cbow.predict(padded_val)
y_pred_lstm_w2v_cbow_val = np.argmax(y_pred, axis=1)
accuracy_lstm_w2v_cbow_val = accuracy_score(y_val, y_pred_lstm_w2v_cbow_val)
precision_lstm_w2v_cbow_val = precision_score(y_val, y_pred_lstm_w2v_cbow_val, average='weighted')
recall_lstm_w2v_cbow_val = recall_score(y_val, y_pred_lstm_w2v_cbow_val, average='weighted')
f1_lstm_w2v_cbow_val = f1_score(y_val,y_pred_lstm_w2v_cbow_val, average='weighted')
print('Classification report')
print(classification_report(y_val, y_pred_lstm_w2v_cbow_val))
print(f'Accuracy: {accuracy_lstm_w2v_cbow_val:.4f}')
print(f'Precision: {precision_lstm_w2v_cbow_val:.4f}')
print(f'Recall: {recall_lstm_w2v_cbow_val:.4f}')
print(f'F1 Score: {f1_lstm_w2v_cbow_val:.4f}')
w2v_cbow_results['LSTM Accuracy w2v cbow val']=accuracy_lstm_w2v_cbow_val
w2v_cbow_results['LSTM Precision w2v cbow val']=precision_lstm_w2v_cbow_val
w2v_cbow_results['LSTM  Recall w2v cbow val']=recall_lstm_w2v_cbow_val
w2v_cbow_results['LSTM  F1 w2v cbow val']=f1_lstm_w2v_cbow_val


32/32 [==============================] - 0s 3ms/step - loss: 0.5585 - accuracy: 0.8690
Test Accuracy: 0.8690000176429749
32/32 [==============================] - 0s 2ms/step
Classification report
              precision    recall  f1-score   support

           0       0.84      0.66      0.74       180
           1       0.88      0.93      0.91       601
           2       0.87      0.91      0.89       198
           3       0.56      0.48      0.51        21

    accuracy                           0.87      1000
   macro avg       0.79      0.74      0.76      1000
weighted avg       0.87      0.87      0.87      1000

Accuracy: 0.8690
Precision: 0.8663
Recall: 0.8690
F1 Score: 0.8651


In [ ]:
test_loss_lstm_w2v_cbow, test_accuracy_lstm_w2v_cbow = model_LSTM_w2v_cbow.evaluate(padded_test, y_test_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {test_accuracy_lstm_w2v_cbow}')

y_pred_LSTM_w2v_cbow_test =model_LSTM_w2v_cbow.predict(padded_test)
y_pred_LSTM_w2v_cbow_test = np.argmax(y_pred, axis=1)
accuracy_LSTM_w2v_cbow_test = accuracy_score(y_test, y_pred_LSTM_w2v_cbow_test)
precision_LSTM_w2v_cbow_test = precision_score(y_test, y_pred_LSTM_w2v_cbow_test, average='weighted')
recall_LSTM_w2v_cbow_test = recall_score(y_test, y_pred_LSTM_w2v_cbow_test, average='weighted')
f1_LSTM_w2v_cbow_test = f1_score(y_test, y_pred_LSTM_w2v_cbow_test, average='weighted')
print('Classification report')
print(classification_report(y_test, y_pred_LSTM_w2v_cbow_test))
print(f'Accuracy: {accuracy_LSTM_w2v_cbow_test:.4f}')
print(f'Precision: {precision_LSTM_w2v_cbow_test:.4f}')
print(f'Recall: {recall_LSTM_w2v_cbow_test:.4f}')
print(f'F1 Score: {f1_LSTM_w2v_cbow_test:.4f}')
w2v_cbow_results['LSTM Accuracy w2v cbow test']=accuracy_LSTM_w2v_cbow_test
w2v_cbow_results['LSTM Precision w2v cbow test']=precision_LSTM_w2v_cbow_test
w2v_cbow_results['LSTM  Recall w2v cbow test']=recall_LSTM_w2v_cbow_test
w2v_cbow_results['LSTM  F1 w2v cbow test']=f1_LSTM_w2v_cbow_test

32/32 [==============================] - 0s 3ms/step - loss: 0.8991 - accuracy: 0.7830
Test Accuracy: 0.7829999923706055
32/32 [==============================] - 0s 2ms/step
Classification report
              precision    recall  f1-score   support

           0       0.18      0.12      0.15       200
           1       0.52      0.64      0.57       512
           2       0.17      0.17      0.17       200
           3       0.11      0.02      0.04        88

    accuracy                           0.39      1000
   macro avg       0.24      0.24      0.23      1000
weighted avg       0.34      0.39      0.36      1000

Accuracy: 0.3900
Precision: 0.3437
Recall: 0.3900
F1 Score: 0.3600


#### RNN

In [ ]:
model_rnn_w2v_cbow=Sequential()
model_rnn_w2v_cbow.add(Embedding(input_dim=total_words,
                    output_dim=300,
                    input_length=max_sequence_len
                    ,weights=[embedding_weight_matrix_w2v_cbow],trainable=False))
model_rnn_w2v_cbow.add(SimpleRNN(100))
model_rnn_w2v_cbow.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_rnn_w2v_cbow.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_rnn_w2v_cbow.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)

Epoch 1/10
125/125 [==============================] - 3s 12ms/step - loss: 0.8168 - accuracy: 0.6877
Epoch 2/10
125/125 [==============================] - 2s 14ms/step - loss: 0.6092 - accuracy: 0.7602
Epoch 3/10
125/125 [==============================] - 3s 21ms/step - loss: 0.5499 - accuracy: 0.7870
Epoch 4/10
125/125 [==============================] - 3s 23ms/step - loss: 0.5087 - accuracy: 0.7968
Epoch 5/10
125/125 [==============================] - 3s 21ms/step - loss: 0.4946 - accuracy: 0.7997
Epoch 6/10
125/125 [==============================] - 2s 12ms/step - loss: 0.4852 - accuracy: 0.8112
Epoch 7/10
125/125 [==============================] - 2s 13ms/step - loss: 0.4665 - accuracy: 0.8167
Epoch 8/10
125/125 [==============================] - 2s 13ms/step - loss: 0.4636 - accuracy: 0.8180
Epoch 9/10
125/125 [==============================] - 2s 12ms/step - loss: 0.4468 - accuracy: 0.8160
Epoch 10/10
125/125 [==============================] - 2s 12ms/step - loss: 0.4283 - accura

In [ ]:
val_loss_rnn_w2v_cbow, val_accuracy_rnn_w2v_cbow= model_rnn_w2v_cbow.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {val_accuracy_rnn_w2v_cbow}')

y_pred = model_rnn_w2v_cbow.predict(padded_val)
y_pred_rnn_w2v_cbow_val = np.argmax(y_pred, axis=1)
accuracy_rnn_w2v_cbow_val = accuracy_score(y_val, y_pred_rnn_w2v_cbow_val)
precision_rnn_w2v_cbow_val = precision_score(y_val, y_pred_rnn_w2v_cbow_val, average='weighted')
recall_rnn_w2v_cbow_val = recall_score(y_val, y_pred_rnn_w2v_cbow_val, average='weighted')
f1_rnn_w2v_cbow_val = f1_score(y_val, y_pred_rnn_w2v_cbow_val, average='weighted')
print('Classification report')
print(classification_report(y_val, y_pred_rnn_w2v_cbow_val))
print(f'Accuracy: {accuracy_rnn_w2v_cbow_val:.4f}')
print(f'Precision: {precision_rnn_w2v_cbow_val:.4f}')
print(f'Recall: {recall_rnn_w2v_cbow_val:.4f}')
print(f'F1 Score: {f1_rnn_w2v_cbow_val:.4f}')
w2v_cbow_results['RNN Accuracy w2v cbow val']=accuracy_rnn_w2v_cbow_val
w2v_cbow_results['RNN Precision w2v cbow val']=precision_rnn_w2v_cbow_val
w2v_cbow_results['RNN  Recall w2v cbow val']=recall_rnn_w2v_cbow_val
w2v_cbow_results['RNN  F1 w2v cbow val']=f1_rnn_w2v_cbow_val

32/32 [==============================] - 0s 7ms/step - loss: 0.7109 - accuracy: 0.7550
Test Accuracy: 0.7549999952316284
32/32 [==============================] - 0s 5ms/step
Classification report
              precision    recall  f1-score   support

           0       0.62      0.43      0.51       180
           1       0.80      0.88      0.83       601
           2       0.77      0.72      0.74       198
           3       0.29      0.43      0.35        21

    accuracy                           0.76      1000
   macro avg       0.62      0.61      0.61      1000
weighted avg       0.75      0.76      0.75      1000

Accuracy: 0.7550
Precision: 0.7501
Recall: 0.7550
F1 Score: 0.7479


In [ ]:
test_loss_rnn_w2v_cbow, test_accuracy_rnn_w2v_cbow = model_rnn_w2v_cbow.evaluate(padded_test, y_test_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {test_accuracy_rnn_w2v_cbow}')
y_pred = model_rnn_w2v_cbow.predict(padded_test)
y_pred_rnn_w2v_cbow_test = np.argmax(y_pred, axis=1)
accuracy_rnn_w2v_cbow_test = accuracy_score(y_test, y_pred_rnn_w2v_cbow_test)
precision_rnn_w2v_cbow_test = precision_score(y_test, y_pred_rnn_w2v_cbow_test, average='weighted')
recall_rnn_w2v_cbow_test = recall_score(y_test, y_pred_rnn_w2v_cbow_test, average='weighted')
f1_rnn_w2v_cbow_test = f1_score(y_test, y_pred_rnn_w2v_cbow_test, average='weighted')
print('Classification report')
print(classification_report(y_test, y_pred_rnn_w2v_cbow_test))
print(f'Accuracy: {accuracy_rnn_w2v_cbow_test:.4f}')
print(f'Precision: {precision_rnn_w2v_cbow_test:.4f}')
print(f'Recall: {recall_rnn_w2v_cbow_test:.4f}')
print(f'F1 Score: {f1_rnn_w2v_cbow_test:.4f}')
w2v_cbow_results['RNN Accuracy w2v cbow test']=accuracy_rnn_w2v_cbow_test
w2v_cbow_results['RNN Precision w2v cbow test']=precision_rnn_w2v_cbow_test
w2v_cbow_results['RNN  Recall w2v cbow test']=recall_rnn_w2v_cbow_test
w2v_cbow_results['RNN  F1 w2v cbow test']=f1_rnn_w2v_cbow_test

32/32 [==============================] - 0s 6ms/step - loss: 0.9837 - accuracy: 0.6730
Test Accuracy: 0.6729999780654907
32/32 [==============================] - 0s 5ms/step
Classification report
              precision    recall  f1-score   support

           0       0.63      0.37      0.47       200
           1       0.68      0.88      0.77       512
           2       0.70      0.65      0.67       200
           3       0.53      0.22      0.31        88

    accuracy                           0.67      1000
   macro avg       0.64      0.53      0.55      1000
weighted avg       0.66      0.67      0.65      1000

Accuracy: 0.6730
Precision: 0.6617
Recall: 0.6730
F1 Score: 0.6478


### evaluation word2vec CBOW

In [ ]:
w2v_cbow_results_list=[{'performance metric & model':k,"results":v} for k,v in w2v_cbow_results.items()]

w2v_cbow_df_results=pd.DataFrame(w2v_cbow_results_list)
w2v_cbow_df_results

,performance metric & model,results
0,Naive Bayes Accuracy w2v cbow val,0.865000
1,Naive Bayes Precision w2v cbow val,0.879618
2,Naive Bayes Recall w2v cbow val,0.865000
3,Naive Bayes F1 w2v cbow val,0.870177
4,Naive Bayes Accuracy w2v cbow test,0.360000
5,Naive Bayes Precision w2v cbow test,0.331769
6,Naive Bayes Recall w2v cbow test,0.360000
7,Naive Bayes F1 w2v cbow test,0.344390
8,Logistic Regression Accuracy w2v cbow val,0.905000
9,Logistic Regression Precision w2v cbow val,0.903901


## Wor2vec twitter skipgram 300

In [ ]:
sg_w2v_results={}

In [ ]:
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_sg_300_twitter.zip"

--2024-06-11 22:02:38--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_sg_300_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3322543144 (3.1G) [application/zip]
Saving to: ‘full_grams_sg_300_twitter.zip’

full_grams_sg_300_t 100%[===================>]   3.09G  22.3MB/s    in 2m 28s  

2024-06-11 22:05:07 (21.4 MB/s) - ‘full_grams_sg_300_twitter.zip’ saved [3322543144/3322543144]



In [ ]:
!unzip "/content/full_grams_sg_300_twitter.zip"

Archive:  /content/full_grams_sg_300_twitter.zip
  inflating: full_grams_sg_300_twitter.mdl  
  inflating: full_grams_sg_300_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_grams_sg_300_twitter.mdl.wv.vectors.npy  


In [ ]:
w2v_model_sg = gensim.models.Word2Vec.load('/content/full_grams_sg_300_twitter.mdl')

In [ ]:
X_train_w2v_embeddings_sg = train_df['processed_text'].apply(lambda sent: get_doc_vec(sent, w2v_model_sg,'train'))
X_val_w2v_embedding_sg = val_df['processed_text'].apply(lambda sent: get_doc_vec(sent, w2v_model_sg,'val'))

X_test_w2v_embeddings_sg = df_test['processed_text'].apply(lambda sent: get_doc_vec(sent, w2v_model_sg,'test'))

In [ ]:
X_train_w2v_embeddings_list_sg = []
for embedding in X_train_w2v_embeddings_sg:
    if embedding is not None:
        X_train_w2v_embeddings_list_sg.append(embedding)
    else:
        # Handle cases where the embedding is None
        X_train_w2v_embeddings_list_sg.append(np.zeros_like(X_train_w2v_embeddings_sg[0]))

X_val_w2v_embeddings_list_sg = []
for embedding in X_val_w2v_embedding_sg:
    if embedding is not None:
        X_val_w2v_embeddings_list_sg.append(embedding)
    else:
        # Handle cases where the embedding is None
        X_val_w2v_embeddings_list_sg.append(np.zeros_like(X_val_w2v_embedding_sg[0]))

X_test_w2v_embeddings_list_sg = []
for embedding in X_test_w2v_embeddings_sg:
    if embedding is not None:
        X_test_w2v_embeddings_list_sg.append(embedding)
    else:
        # Handle cases where the embedding is None
        X_test_w2v_embeddings_list_sg.append(np.zeros_like(X_test_w2v_embeddings_sg[0]))


### Traditional models

#### Naive Bayes

In [ ]:
clf_nb_w2v_sg= GaussianNB()
clf_nb_w2v_sg.fit(X_train_w2v_embeddings_list_sg,train_df['Type'])

y_pred_nb_w2v_sg_val = clf_nb_w2v_sg.predict(X_val_w2v_embeddings_list_sg)

accuracy_nb_w2v_sg_val = accuracy_score(val_df['Type'], y_pred_nb_w2v_sg_val)
precision_nb_w2v_sg_val = precision_score(val_df['Type'], y_pred_nb_w2v_sg_val, average='weighted')
recall_nb_w2v_sg_val = recall_score(val_df['Type'], y_pred_nb_w2v_sg_val, average='weighted')
f1_nb_w2v_sg_val = f1_score(val_df['Type'], y_pred_nb_w2v_sg_val, average='weighted')

print(f"Accuracy validation: {accuracy_nb_w2v_sg_val:.2f}")
print(f"Precision validation: {precision_nb_w2v_sg_val:.2f}")
print(f"Recall validation: {recall_nb_w2v_sg_val:.2f}")
print(f"F1 Score validation: {f1_nb_w2v_sg_val:.2f}")
print("Classification Report for validation:")
print(classification_report(val_df['Type'], y_pred_nb_w2v_sg_val))
sg_w2v_results['Naive Bayes Accuracy w2v sg val']=accuracy_nb_w2v_sg_val
sg_w2v_results['Naive Bayes Precision w2v sg val']=precision_nb_w2v_sg_val
sg_w2v_results['Naive Bayes  Recall w2v sg val']=recall_nb_w2v_sg_val
sg_w2v_results['Naive Bayes  F1 w2v sg val']=f1_nb_w2v_sg_val

Accuracy validation: 0.88
Precision validation: 0.89
Recall validation: 0.88
F1 Score validation: 0.88
Classification Report for validation:
              precision    recall  f1-score   support

    economic       0.75      0.79      0.77       180
    politics       0.91      0.91      0.91       601
       sport       0.97      0.87      0.92       198
        tech       0.53      0.86      0.65        21

    accuracy                           0.88      1000
   macro avg       0.79      0.86      0.81      1000
weighted avg       0.89      0.88      0.88      1000



In [ ]:
y_pred_nb_w2v_sg_test = clf_nb_w2v_sg.predict(X_test_w2v_embeddings_list_sg)

accuracy_nb_w2v_sg_test = accuracy_score(df_test['Type'], y_pred_nb_w2v_sg_test)
precision_nb_w2v_sg_test = precision_score(df_test['Type'], y_pred_nb_w2v_sg_test, average='weighted')
recall_nb_w2v_sg_test = recall_score(df_test['Type'], y_pred_nb_w2v_sg_test, average='weighted')
f1_nb_w2v_sg_test = f1_score(df_test['Type'], y_pred_nb_w2v_sg_test, average='weighted')

print(f"Accuracy: {accuracy_nb_w2v_sg_test:.2f}")
print(f"Precision : {precision_nb_w2v_sg_test:.2f}")
print(f"Recall : {recall_nb_w2v_sg_test:.2f}")
print(f"F1 Score : {f1_nb_w2v_sg_test:.2f}")
print("Classification Report for test:")
print(classification_report(df_test['Type'], y_pred_nb_w2v_sg_test))

sg_w2v_results['Naive Bayes Accuracy w2v sg test']=accuracy_nb_w2v_sg_test
sg_w2v_results['Naive Bayes Precision w2v sg test']=precision_nb_w2v_sg_test
sg_w2v_results['Naive Bayes  Recall w2v sg test']=recall_nb_w2v_sg_test
sg_w2v_results['Naive Bayes  F1 w2v sg test']=f1_nb_w2v_sg_test

Accuracy: 0.83
Precision : 0.83
Recall : 0.83
F1 Score : 0.83
Classification Report for test:
              precision    recall  f1-score   support

    economic       0.70      0.70      0.70       200
    politics       0.84      0.88      0.86       512
       sport       0.99      0.92      0.95       200
        tech       0.71      0.64      0.67        88

    accuracy                           0.83      1000
   macro avg       0.81      0.78      0.80      1000
weighted avg       0.83      0.83      0.83      1000



"\nsg_w2v_results['Naive Bayes Accuracy w2v sg test']=accuracy_nb_w2v_sg_test\nsg_w2v_results['Naive Bayes Precision w2v sg test']=precision_nb_w2v_sg_test\nsg_w2v_results['Naive Bayes  Recall w2v sg test']=recall_nb_w2v_sg_test\nsg_w2v_results['Naive Bayes  F1 w2v sg test']=f1_nb_w2v_sg_test"

#### Logestic Regression

In [ ]:
clf_lr_w2v_sg = LogisticRegression(random_state=0)
clf_lr_w2v_sg.fit(X_train_w2v_embeddings_list_sg, train_df['Type'])

y_pred_lr_w2v_sg_val = clf_lr_w2v_sg.predict(X_val_w2v_embeddings_list_sg)

accuracy_lr_w2v_sg_val = accuracy_score(val_df['Type'], y_pred_lr_w2v_sg_val)
precision_lr_w2v_sg_val = precision_score(val_df['Type'], y_pred_lr_w2v_sg_val, average='weighted')
recall_lr_w2v_sg_val = recall_score(val_df['Type'], y_pred_lr_w2v_sg_val, average='weighted')
f1_lr_w2v_sg_val = f1_score(val_df['Type'], y_pred_lr_w2v_sg_val, average='weighted')

print(f"Accuracy validation: {accuracy_lr_w2v_sg_val:.2f}")
print(f"Precision validation: {precision_lr_w2v_sg_val:.2f}")
print(f"Recall validation: {recall_lr_w2v_sg_val:.2f}")
print(f"F1 Score validation: {f1_lr_w2v_sg_val:.2f}")
print("Classification Report for validation:")
print(classification_report(val_df['Type'], y_pred_lr_w2v_sg_val))



sg_w2v_results['Logistic Regression Accuracy w2v sg val']=accuracy_lr_w2v_sg_val
sg_w2v_results['Logistic Regression Precision w2v sg val']=precision_lr_w2v_sg_val
sg_w2v_results['Logistic Regression  Recall w2v sg val']=recall_lr_w2v_sg_val
sg_w2v_results['Logistic Regression  F1 w2v sg val']=f1_lr_w2v_sg_val


Accuracy validation: 0.91
Precision validation: 0.91
Recall validation: 0.91
F1 Score validation: 0.91
Classification Report for validation:
              precision    recall  f1-score   support

    economic       0.88      0.76      0.82       180
    politics       0.90      0.97      0.93       601
       sport       0.97      0.91      0.94       198
        tech       0.93      0.62      0.74        21

    accuracy                           0.91      1000
   macro avg       0.92      0.81      0.86      1000
weighted avg       0.91      0.91      0.91      1000



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
y_pred_lr_w2v_sg_test = clf_lr_w2v_sg.predict(X_test_w2v_embeddings_list_sg)

accuracy_lr_w2v_sg_test = accuracy_score(df_test['Type'], y_pred_lr_w2v_sg_test)
precision_lr_w2v_sg_test = precision_score(df_test['Type'], y_pred_lr_w2v_sg_test, average='weighted')
recall_lr_w2v_sg_test = recall_score(df_test['Type'], y_pred_lr_w2v_sg_test, average='weighted')
f1_lr_w2v_sg_test = f1_score(df_test['Type'], y_pred_lr_w2v_sg_test, average='weighted')

print(f"Accuracy : {accuracy_lr_w2v_sg_test:.2f}")
print(f"Precision : {precision_lr_w2v_sg_test:.2f}")
print(f"Recall : {recall_lr_w2v_sg_test:.2f}")
print(f"F1 Score : {f1_lr_w2v_sg_test:.2f}")
print("Classification Report for :")
print(classification_report(df_test['Type'], y_pred_lr_w2v_sg_test))

sg_w2v_results['Logistic Regression Accuracy w2v sg test']=accuracy_lr_w2v_sg_test
sg_w2v_results['Logistic Regression Precision w2v sg test']=precision_lr_w2v_sg_test
sg_w2v_results['Logistic Regression  Recall w2v sg test']=recall_lr_w2v_sg_test
sg_w2v_results['Logistic Regression  F1 w2v sg test']=f1_lr_w2v_sg_test


Accuracy : 0.84
Precision : 0.85
Recall : 0.84
F1 Score : 0.83
Classification Report for :
              precision    recall  f1-score   support

    economic       0.77      0.70      0.73       200
    politics       0.82      0.94      0.88       512
       sport       0.97      0.95      0.96       200
        tech       0.89      0.38      0.53        88

    accuracy                           0.84      1000
   macro avg       0.86      0.74      0.77      1000
weighted avg       0.85      0.84      0.83      1000



"\nsg_w2v_results['Logistic Regression Accuracy w2v sg test']=accuracy_lr_w2v_sg_test\nsg_w2v_results['Logistic Regression Precision w2v sg test']=precision_lr_w2v_sg_test\nsg_w2v_results['Logistic Regression  Recall w2v sg test']=recall_lr_w2v_sg_test\nsg_w2v_results['Logistic Regression  F1 w2v sg test']=f1_lr_w2v_sg_test"

### Neural Network

#### embedding weights

In [ ]:
embedding_weight_matrix_w2v_sg=np.zeros((total_words,300))

for word, i in tokenizer.word_index.items():
  if word in w2v_model_sg.wv:
    embedding_weight_matrix_w2v_sg[i]=w2v_model_sg.wv[word]
  else:
    embedding_weight_matrix_w2v_sg[i]=np.random.uniform(-25,25)


#### LSTM

In [ ]:
model_LSTM_w2v_sg=Sequential()
model_LSTM_w2v_sg.add(Embedding(input_dim=total_words,
                    output_dim=300,
                    input_length=max_sequence_len
                    ,weights=[embedding_weight_matrix_w2v_sg],trainable=False))
model_LSTM_w2v_sg.add(LSTM(100))
model_LSTM_w2v_sg.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_LSTM_w2v_sg.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_LSTM_w2v_sg.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)




Epoch 1/10
125/125 [==============================] - 2s 4ms/step - loss: 0.4425 - accuracy: 0.8462
Epoch 2/10
125/125 [==============================] - 1s 4ms/step - loss: 0.2511 - accuracy: 0.9147
Epoch 3/10
125/125 [==============================] - 1s 4ms/step - loss: 0.1620 - accuracy: 0.9467
Epoch 4/10
125/125 [==============================] - 1s 4ms/step - loss: 0.1163 - accuracy: 0.9590
Epoch 5/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0598 - accuracy: 0.9820
Epoch 6/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0359 - accuracy: 0.9900
Epoch 7/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0391 - accuracy: 0.9883
Epoch 8/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0507 - accuracy: 0.9830
Epoch 9/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0426 - accuracy: 0.9852
Epoch 10/10
125/125 [==============================] - 1s 4ms/step - loss: 0.0521 - accuracy: 0.9833

In [ ]:
val_loss_lstm_w2v_sg, val_accuracy_lstm_w2v_sg= model_LSTM_w2v_sg.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'validation Accuracy: {val_accuracy_lstm_w2v_sg}')
y_pred = model_LSTM_w2v_sg.predict(padded_val)
y_pred_lstm_w2v_sg_val = np.argmax(y_pred, axis=1)
accuracy_lstm_w2v_sg_val = accuracy_score(y_val, y_pred_lstm_w2v_sg_val)
precision_lstm_w2v_sg_val = precision_score(y_val, y_pred_lstm_w2v_sg_val, average='weighted')
recall_lstm_w2v_sg_val = recall_score(y_val, y_pred_lstm_w2v_sg_val, average='weighted')
f1_lstm_w2v_sg_val = f1_score(y_val,y_pred_lstm_w2v_sg_val, average='weighted')
print("Classification report")
print(classification_report(y_val, y_pred_lstm_w2v_sg_val))
print(f'Accuracy: {accuracy_lstm_w2v_sg_val:.4f}')
print(f'Precision: {precision_lstm_w2v_sg_val:.4f}')
print(f'Recall: {recall_lstm_w2v_sg_val:.4f}')
print(f'F1 Score: {f1_lstm_w2v_sg_val:.4f}')
sg_w2v_results['LSTM Accuracy w2v sg val']=accuracy_lstm_w2v_sg_val
sg_w2v_results['LSTM Precision w2v sg val']=precision_lstm_w2v_sg_val
sg_w2v_results['LSTM  Recall w2v sg val']=recall_lstm_w2v_sg_val
sg_w2v_results['LSTM  F1 w2v sg val']=f1_lstm_w2v_sg_val


32/32 [==============================] - 1s 3ms/step - loss: 0.4592 - accuracy: 0.8900
validation Accuracy: 0.8899999856948853
32/32 [==============================] - 0s 2ms/step
Classification report
              precision    recall  f1-score   support

           0       0.74      0.83      0.78       180
           1       0.93      0.93      0.93       601
           2       0.96      0.87      0.91       198
           3       0.47      0.43      0.45        21

    accuracy                           0.89      1000
   macro avg       0.78      0.76      0.77      1000
weighted avg       0.89      0.89      0.89      1000

Accuracy: 0.8900
Precision: 0.8932
Recall: 0.8900
F1 Score: 0.8908


In [ ]:
test_loss_lstm_w2v_sg, test_accuracy_lstm_w2v_sg = model_LSTM_w2v_sg.evaluate(padded_test, y_test_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {test_accuracy_lstm_w2v_sg}')
y_pred = model_LSTM_w2v_sg.predict(padded_test)
y_pred_lstm_w2v_sg_test = np.argmax(y_pred, axis=1)
accuracy_lstm_w2v_sg_test = accuracy_score(y_test, y_pred_lstm_w2v_sg_test)
precision_lstm_w2v_sg_test = precision_score(y_test, y_pred_lstm_w2v_sg_test, average='weighted')
recall_lstm_w2v_sg_test = recall_score(y_test, y_pred_lstm_w2v_sg_test, average='weighted')
f1_lstm_w2v_sg_test = f1_score(y_test,y_pred_lstm_w2v_sg_test, average='weighted')
print("Classification report")
print(classification_report(y_test, y_pred_lstm_w2v_sg_test))
print(f'Accuracy: {accuracy_lstm_w2v_sg_test:.4f}')
print(f'Precision: {precision_lstm_w2v_sg_test:.4f}')
print(f'Recall: {recall_lstm_w2v_sg_test:.4f}')
print(f'F1 Score: {f1_lstm_w2v_sg_test:.4f}')
sg_w2v_results['LSTM Accuracy w2v sg test']=accuracy_lstm_w2v_sg_test
sg_w2v_results['LSTM Precision w2v sg test']=precision_lstm_w2v_sg_test
sg_w2v_results['LSTM  Recall w2v sg test']=recall_lstm_w2v_sg_test
sg_w2v_results['LSTM  F1 w2v sg test']=f1_lstm_w2v_sg_test

32/32 [==============================] - 0s 3ms/step - loss: 0.8517 - accuracy: 0.7990
Test Accuracy: 0.7990000247955322
32/32 [==============================] - 0s 2ms/step
Classification report
              precision    recall  f1-score   support

           0       0.59      0.76      0.66       200
           1       0.84      0.85      0.85       512
           2       0.95      0.93      0.94       200
           3       0.86      0.28      0.43        88

    accuracy                           0.80      1000
   macro avg       0.81      0.71      0.72      1000
weighted avg       0.81      0.80      0.79      1000

Accuracy: 0.7990
Precision: 0.8144
Recall: 0.7990
F1 Score: 0.7918


#### RNN

In [ ]:
model_rnn_w2v_sg=Sequential()
model_rnn_w2v_sg.add(Embedding(input_dim=total_words,
                    output_dim=300,
                    input_length=max_sequence_len
                    ,weights=[embedding_weight_matrix_w2v_sg],trainable=False))
model_rnn_w2v_sg.add(SimpleRNN(100))
model_rnn_w2v_sg.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_rnn_w2v_sg.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_rnn_w2v_sg.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)

Epoch 1/10
125/125 [==============================] - 3s 18ms/step - loss: 0.8384 - accuracy: 0.6877
Epoch 2/10
125/125 [==============================] - 3s 21ms/step - loss: 0.6210 - accuracy: 0.7648
Epoch 3/10
125/125 [==============================] - 2s 13ms/step - loss: 0.5753 - accuracy: 0.7745
Epoch 4/10
125/125 [==============================] - 2s 13ms/step - loss: 0.5596 - accuracy: 0.7947
Epoch 5/10
125/125 [==============================] - 2s 13ms/step - loss: 0.5233 - accuracy: 0.7997
Epoch 6/10
125/125 [==============================] - 2s 12ms/step - loss: 0.4773 - accuracy: 0.8202
Epoch 7/10
125/125 [==============================] - 2s 12ms/step - loss: 0.4809 - accuracy: 0.8180
Epoch 8/10
125/125 [==============================] - 2s 12ms/step - loss: 0.4822 - accuracy: 0.8108
Epoch 9/10
125/125 [==============================] - 2s 18ms/step - loss: 0.4758 - accuracy: 0.8145
Epoch 10/10
125/125 [==============================] - 3s 21ms/step - loss: 0.4475 - accura

In [ ]:
val_loss_rnn_w2v_sg, val_accuracy_lstm_w2v_sg= model_rnn_w2v_sg.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {val_accuracy_lstm_w2v_sg}')
y_pred = model_rnn_w2v_sg.predict(padded_val)
y_pred_rnn_w2v_sg_val = np.argmax(y_pred, axis=1)
accuracy_rnn_w2v_sg_val = accuracy_score(y_val, y_pred_rnn_w2v_sg_val )
precision_rnn_w2v_sg_val = precision_score(y_val, y_pred_rnn_w2v_sg_val , average='weighted')
recall_rnn_w2v_sg_val = recall_score(y_val, y_pred_rnn_w2v_sg_val , average='weighted')
f1_rnn_w2v_sg_val = f1_score(y_val,y_pred_rnn_w2v_sg_val , average='weighted')
print('Classification report')
print(classification_report(y_val, y_pred_rnn_w2v_sg_val))
print(f'Accuracy: {accuracy_rnn_w2v_sg_val:.4f}')
print(f'Precision: {precision_rnn_w2v_sg_val:.4f}')
print(f'Recall: {recall_rnn_w2v_sg_val:.4f}')
print(f'F1 Score: {f1_rnn_w2v_sg_val:.4f}')
sg_w2v_results['RNN Accuracy w2v sg val']=accuracy_rnn_w2v_sg_val
sg_w2v_results['RNN Precision w2v sg val']=precision_rnn_w2v_sg_val
sg_w2v_results['RNN  Recall w2v sg val']=recall_rnn_w2v_sg_val
sg_w2v_results['RNN  F1 w2v sg val']=f1_rnn_w2v_sg_val

32/32 [==============================] - 0s 4ms/step - loss: 0.6292 - accuracy: 0.7790
Test Accuracy: 0.7789999842643738
32/32 [==============================] - 0s 3ms/step
Classification report
              precision    recall  f1-score   support

           0       0.62      0.58      0.60       180
           1       0.81      0.89      0.85       601
           2       0.91      0.65      0.76       198
           3       0.35      0.52      0.42        21

    accuracy                           0.78      1000
   macro avg       0.67      0.66      0.66      1000
weighted avg       0.79      0.78      0.78      1000

Accuracy: 0.7790
Precision: 0.7868
Recall: 0.7790
F1 Score: 0.7770


In [ ]:
test_loss_lstm_w2v_sg, test_accuracy_lstm_w2v_sg = model_rnn_w2v_sg.evaluate(padded_test, y_test_one_hot_encoded, verbose=1)
print(f'Test Accuracy: {test_accuracy_lstm_w2v_sg}')

y_pred = model_rnn_w2v_sg.predict(padded_test)
y_pred_rnn_w2v_sg_test = np.argmax(y_pred, axis=1)
accuracy_rnn_w2v_sg_test = accuracy_score(y_val, y_pred_rnn_w2v_sg_test )
precision_rnn_w2v_sg_test = precision_score(y_val, y_pred_rnn_w2v_sg_test , average='weighted')
recall_rnn_w2v_sg_test = recall_score(y_val, y_pred_rnn_w2v_sg_test , average='weighted')
f1_rnn_w2v_sg_test = f1_score(y_val,y_pred_rnn_w2v_sg_test , average='weighted')
print('Classification report')
print(classification_report(y_test, y_pred_rnn_w2v_sg_test))
print(f'Accuracy: {accuracy_rnn_w2v_sg_test:.4f}')
print(f'Precision: {precision_rnn_w2v_sg_test:.4f}')
print(f'Recall: {recall_rnn_w2v_sg_test:.4f}')
print(f'F1 Score: {f1_rnn_w2v_sg_test:.4f}')
sg_w2v_results['RNN Accuracy w2v sg test']=accuracy_rnn_w2v_sg_test
sg_w2v_results['RNN Precision w2v sg test']=precision_rnn_w2v_sg_test
sg_w2v_results['RNN  Recall w2v sg test']=recall_rnn_w2v_sg_test
sg_w2v_results['RNN  F1 w2v sg test']=f1_rnn_w2v_sg_test

32/32 [==============================] - 0s 4ms/step - loss: 0.9862 - accuracy: 0.6900
Test Accuracy: 0.6899999976158142
32/32 [==============================] - 0s 4ms/step
Classification report
              precision    recall  f1-score   support

           0       0.58      0.47      0.52       200
           1       0.69      0.90      0.78       512
           2       0.87      0.61      0.72       200
           3       0.52      0.14      0.22        88

    accuracy                           0.69      1000
   macro avg       0.66      0.53      0.56      1000
weighted avg       0.69      0.69      0.67      1000

Accuracy: 0.4450
Precision: 0.4133
Recall: 0.4450
F1 Score: 0.4272


#### Evaluation of Word2Vec skipgram

In [ ]:
sg_w2v_results_list=[{'performance metric & model':k,"results":v} for k,v in sg_w2v_results.items()]
sg_w2v_df_results=pd.DataFrame(sg_w2v_results_list)
sg_w2v_df_results

,performance metric & model,results
0,Naive Bayes Accuracy w2v sg val,0.880000
1,Naive Bayes Precision w2v sg val,0.887769
2,Naive Bayes Recall w2v sg val,0.880000
3,Naive Bayes F1 w2v sg val,0.882449
4,Naive Bayes Accuracy w2v sg test,0.832000
5,Naive Bayes Precision w2v sg test,0.832979
6,Naive Bayes Recall w2v sg test,0.832000
7,Naive Bayes F1 w2v sg test,0.831797
8,Logistic Regression Accuracy w2v sg val,0.911000
9,Logistic Regression Precision w2v sg val,0.911339


# evaluation
#### note that the evaluation will be in another notebook

In [ ]:
evaluation_df=pd.concat([tf_df_results,bow_df_results,w2v_cbow_df_results,sg_w2v_df_results],ignore_index=True)

In [ ]:
evaluation_df.to_csv('evaluation_df_4.csv',index=False)

In [ ]:
evaluation_df

,performance metric & model,results
0,Naive Bayes Accuracy tf val,0.409000
1,Naive Bayes Precision tf val,0.516221
2,Naive Bayes Recall tf val,0.409000
3,Naive Bayes F1 tf val,0.441906
4,Naive Bayes Accuracy tf test,0.387000
...,...,...
123,RNN F1 w2v sg val,0.777012
124,RNN Accuracy w2v sg test,0.445000
125,RNN Precision w2v sg test,0.413297
126,RNN Recall w2v sg test,0.445000
